In [55]:
import pandas as pd
import numpy as np

from tensorflow import keras
from sklearn.model_selection import train_test_split

Datos obtenidos de: https://www.kaggle.com/code/tmishinev/nlp-depression-tweets-keras-lstm

In [2]:
raw = pd.read_csv("Mental-Health-Twitter.csv")
raw.shape

(20000, 11)

In [3]:
raw.head(10)

,Unnamed: 0,post_id,post_created,post_text,user_id,followers,friends,favourites,statuses,retweets,label
0,0,637894677824413696,Sun Aug 30 07:48:37 +0000 2015,It's just over 2 years since I was diagnosed w...,1013187241,84,211,251,837,0,1
1,1,637890384576778240,Sun Aug 30 07:31:33 +0000 2015,"It's Sunday, I need a break, so I'm planning t...",1013187241,84,211,251,837,1,1
2,2,637749345908051968,Sat Aug 29 22:11:07 +0000 2015,Awake but tired. I need to sleep but my brain ...,1013187241,84,211,251,837,0,1
3,3,637696421077123073,Sat Aug 29 18:40:49 +0000 2015,RT @SewHQ: #Retro bears make perfect gifts and...,1013187241,84,211,251,837,2,1
4,4,637696327485366272,Sat Aug 29 18:40:26 +0000 2015,It’s hard to say whether packing lists are mak...,1013187241,84,211,251,837,1,1
5,5,637692793083817985,Sat Aug 29 18:26:24 +0000 2015,Making packing lists is my new hobby... #movin...,1013187241,84,211,251,837,1,1
6,6,637691649943072772,Sat Aug 29 18:21:51 +0000 2015,At what point does keeping stuff for nostalgic...,1013187241,84,211,251,837,1,1
7,7,637689418472652800,Sat Aug 29 18:12:59 +0000 2015,Currently in the finding-boxes-of-random-shit ...,1013187241,84,211,251,837,0,1
8,8,637687177946734592,Sat Aug 29 18:04:05 +0000 2015,"Can't be bothered to cook, take away on the wa...",1013187241,84,211,251,837,0,1
9,9,637684866906255360,Sat Aug 29 17:54:54 +0000 2015,RT @itventsnews: ITV releases promo video for ...,1013187241,84,211,251,837,41,1


Para propositos de este proyecto, unicamente interesa el texto del post y el label para realizar clasificacion, por lo cual el dataset se reduce unicamente a estas columnas. Sin embargo, para un futuro analisis se podria realizar un modelo que, dado un handle de Twitter, realice webscrapping y analice los tweets del usuario para determinar si tiene depresion o no. Para este proyecto, unicamente se realizara clasificacion en base a un texto.

In [4]:
data = raw[["post_text", "label"]]
data.head(10)

,post_text,label
0,It's just over 2 years since I was diagnosed w...,1
1,"It's Sunday, I need a break, so I'm planning t...",1
2,Awake but tired. I need to sleep but my brain ...,1
3,RT @SewHQ: #Retro bears make perfect gifts and...,1
4,It’s hard to say whether packing lists are mak...,1
5,Making packing lists is my new hobby... #movin...,1
6,At what point does keeping stuff for nostalgic...,1
7,Currently in the finding-boxes-of-random-shit ...,1
8,"Can't be bothered to cook, take away on the wa...",1
9,RT @itventsnews: ITV releases promo video for ...,1


# Normalizacion
(Normalmente la eliminacion de stop words, junto con lemmatizacion o stemmizacion son parte de la normalizacion del texto, pero planteo la hipotesis de que para un modelo recurrente, la informacion de tener un stop word o de tener un verbo conjugado puede ser mas valioso que solo los stems o lemmas, por lo que no se eliminan)

In [5]:
data["norm_text"] = data["post_text"].str.replace(r'[^a-zA-Z0-9\s{1}áéíóúüñÁÉÍÓÚÑ]', '')
data["norm_text"] = data["norm_text"].str.lower().str.strip().str.rstrip('\n').str.rstrip('\r\n')
data.head(10)

C:\Users\yanco\AppData\Local\Temp\ipykernel_16248\1178538166.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  data["norm_text"] = data["post_text"].str.replace(r'[^a-zA-Z0-9\s{1}áéíóúüñÁÉÍÓÚÑ]', '')
C:\Users\yanco\AppData\Local\Temp\ipykernel_16248\1178538166.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["norm_text"] = data["post_text"].str.replace(r'[^a-zA-Z0-9\s{1}áéíóúüñÁÉÍÓÚÑ]', '')
C:\Users\yanco\AppData\Local\Temp\ipykernel_16248\1178538166.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/use

,post_text,label,norm_text
0,It's just over 2 years since I was diagnosed w...,1,its just over 2 years since i was diagnosed wi...
1,"It's Sunday, I need a break, so I'm planning t...",1,its sunday i need a break so im planning to sp...
2,Awake but tired. I need to sleep but my brain ...,1,awake but tired i need to sleep but my brain h...
3,RT @SewHQ: #Retro bears make perfect gifts and...,1,rt sewhq retro bears make perfect gifts and ar...
4,It’s hard to say whether packing lists are mak...,1,its hard to say whether packing lists are maki...
5,Making packing lists is my new hobby... #movin...,1,making packing lists is my new hobby movinghouse
6,At what point does keeping stuff for nostalgic...,1,at what point does keeping stuff for nostalgic...
7,Currently in the finding-boxes-of-random-shit ...,1,currently in the findingboxesofrandomshit pack...
8,"Can't be bothered to cook, take away on the wa...",1,cant be bothered to cook take away on the way ...
9,RT @itventsnews: ITV releases promo video for ...,1,rt itventsnews itv releases promo video for th...


### Tokenizacion

In [14]:
train, test = train_test_split(data[["norm_text", "label"]], test_size=0.2)
print(train.shape)
train.head(10)

(16000, 2)


,norm_text,label
16872,mhardzsali active na active bah\r\n\r\npaytfor...,0
3684,kimberlym06 those are just your shoes lol,1
19401,california college san diego national city htt...,0
17231,tweet and retweet \r\n\r\ngopayt dreamteamyong,0
2511,rt cbs11doug whats it like to be the homeowner...,1
10322,lydiamcrtins well she was yes but she wasnt ju...,0
12857,salon hey joy this morning was a complete fail...,0
13404,marclotter realdonaldtrump mikepence how ma...,0
19423,skinovate skin care clinic httpstcoxpfp5yaij5 ...,0
5101,gasp you hate me another gasp and my fucks ran...,1


In [18]:
tokenizer = keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(train["norm_text"].to_numpy())
train_sequences = tokenizer.texts_to_sequences(train["norm_text"].to_numpy())
test_sequences = tokenizer.texts_to_sequences(test["norm_text"].to_numpy())

words = len(tokenizer.word_index)
words

29805

In [19]:
max_seq = 0
for seq in train_sequences:
    if len(seq) > max_seq:
        max_seq = len(seq)
for seq in test_sequences:
    if len(seq) > max_seq:
        max_seq = len(seq)
        
max_seq

34

In [21]:
x_train = keras.preprocessing.sequence.pad_sequences(train_sequences, maxlen=max_seq)
x_test = keras.preprocessing.sequence.pad_sequences(test_sequences, maxlen=max_seq)
x_train.shape

(16000, 34)

In [30]:
y_train = train["label"].to_numpy()
y_test = test["label"].to_numpy()
y_train.shape

(16000,)

# Construccion de modelo simple

In [42]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 128)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
simple_lstm = keras.Model(inputs, outputs)

simple_lstm.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

simple_lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 34)]              0         
                                                                 
 embedding (Embedding)       (None, 34, 128)           3815168   
                                                                 
 bidirectional (Bidirectiona  (None, 34, 128)          98816     
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 128)              98816     
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 4,012,929
Trainable params: 4,012,929
Non-train

In [43]:
simple_lstm.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

Epoch 1/2
400/400 [==============================] - 44s 92ms/step - loss: 0.4666 - accuracy: 0.7593 - precision: 0.7569 - val_loss: 0.3248 - val_accuracy: 0.8591 - val_precision: 0.8476
Epoch 2/2
400/400 [==============================] - 35s 88ms/step - loss: 0.1256 - accuracy: 0.9533 - precision: 0.9485 - val_loss: 0.3481 - val_accuracy: 0.8481 - val_precision: 0.8540


In [44]:
simple_lstm.evaluate(x_test, y_test)

125/125 [==============================] - 3s 23ms/step - loss: 0.3494 - accuracy: 0.8522 - precision: 0.8515


[0.34942957758903503, 0.8522499799728394, 0.8514562845230103]

Con un modelo relativamente simple hemos logrado nuestro objetivo de obtener un accuracy de 0.85, con un precision bastante similar.

# Experimentando con mas modelos

In [46]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 128)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(32, activation='relu')(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 64s 120ms/step - loss: 0.4700 - accuracy: 0.7530 - precision: 0.7587 - val_loss: 0.3342 - val_accuracy: 0.8456 - val_precision: 0.8868
Epoch 2/2
400/400 [==============================] - 42s 105ms/step - loss: 0.1285 - accuracy: 0.9505 - precision: 0.9454 - val_loss: 0.3445 - val_accuracy: 0.8675 - val_precision: 0.8751
Evalaucion)
125/125 [==============================] - 3s 27ms/step - loss: 0.3594 - accuracy: 0.8515 - precision: 0.8512


[0.3594318926334381, 0.8514999747276306, 0.8512396812438965]

Agregar una capa densa mejoro el accuracy y precision de la validacion, pero empeoro en la prueba, parece indicar un overfitting, tratemos de agregar droput:

In [47]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 128)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 43s 91ms/step - loss: 0.4799 - accuracy: 0.7502 - precision: 0.7546 - val_loss: 0.3308 - val_accuracy: 0.8469 - val_precision: 0.8536
Epoch 2/2
400/400 [==============================] - 39s 98ms/step - loss: 0.1408 - accuracy: 0.9471 - precision: 0.9414 - val_loss: 0.3769 - val_accuracy: 0.8559 - val_precision: 0.8659

Evalaucion
125/125 [==============================] - 4s 28ms/step - loss: 0.3679 - accuracy: 0.8550 - precision: 0.8560


[0.3679468631744385, 0.8550000190734863, 0.8560273051261902]

Se logra mejorar el accuracy de prueba marginalmente, que pasa si se agregan mas neuronas?

In [48]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(64, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 71s 161ms/step - loss: 0.4636 - accuracy: 0.7562 - precision: 0.7611 - val_loss: 0.3189 - val_accuracy: 0.8547 - val_precision: 0.8581
Epoch 2/2
400/400 [==============================] - 57s 144ms/step - loss: 0.1227 - accuracy: 0.9547 - precision: 0.9509 - val_loss: 0.3529 - val_accuracy: 0.8512 - val_precision: 0.8180

Evalaucion
125/125 [==============================] - 4s 31ms/step - loss: 0.3578 - accuracy: 0.8580 - precision: 0.8219


[0.3578365445137024, 0.8579999804496765, 0.8218818306922913]

Vemos un incremento en el accuracy, pero una decremento en el precision, como este modelo va a servir para ayudar la realizacion de diagnosticos, no es particularmente buena idea permitir esta perdida.

Se realiza otra prueba reduciendo las neuronas de la capa densa

In [49]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(32, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 59s 132ms/step - loss: 0.4656 - accuracy: 0.7620 - precision: 0.7465 - val_loss: 0.3516 - val_accuracy: 0.8369 - val_precision: 0.8856
Epoch 2/2
400/400 [==============================] - 52s 129ms/step - loss: 0.1199 - accuracy: 0.9561 - precision: 0.9504 - val_loss: 0.3334 - val_accuracy: 0.8528 - val_precision: 0.8902

Evalaucion
125/125 [==============================] - 4s 30ms/step - loss: 0.3358 - accuracy: 0.8515 - precision: 0.8805


[0.33578750491142273, 0.8514999747276306, 0.8804634213447571]

Esta prueba reduce nuestro accuray marginalmente, pero aumenta nuestro precision considerablemente.

Que pasa si agregamos una nueva capa recurrente?

In [50]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(32))(x)
x = keras.layers.Dense(16, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 77s 171ms/step - loss: 0.5220 - accuracy: 0.7312 - precision: 0.7250 - val_loss: 0.3444 - val_accuracy: 0.8441 - val_precision: 0.8288
Epoch 2/2
400/400 [==============================] - 65s 162ms/step - loss: 0.1674 - accuracy: 0.9391 - precision: 0.9318 - val_loss: 0.3856 - val_accuracy: 0.8609 - val_precision: 0.8959

Evalaucion
125/125 [==============================] - 5s 41ms/step - loss: 0.4035 - accuracy: 0.8512 - precision: 0.8853


[0.4035157859325409, 0.8512499928474426, 0.8852721452713013]

Se ve un incremento marginal en el precision y un decremento marginal en el accuracy. Aunque se debe notar que hasta ahora se ha tratado de  reducir el nivel de neuronas en cada capa, este es el mejor modelo hasta ahora, veamos que pasa si se cambia la politica de reduccion de neuronas.

In [51]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64))(x)
x = keras.layers.Dense(128, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

model.fit(x_train, y_train, batch_size=32, epochs=2, validation_split=0.2)

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/2
400/400 [==============================] - 81s 177ms/step - loss: 0.4639 - accuracy: 0.7579 - precision: 0.7466 - val_loss: 0.3186 - val_accuracy: 0.8566 - val_precision: 0.8419
Epoch 2/2
400/400 [==============================] - 64s 159ms/step - loss: 0.1251 - accuracy: 0.9529 - precision: 0.9452 - val_loss: 0.3470 - val_accuracy: 0.8606 - val_precision: 0.8589

Evalaucion
125/125 [==============================] - 5s 40ms/step - loss: 0.3523 - accuracy: 0.8615 - precision: 0.8524


[0.3523355722427368, 0.8615000247955322, 0.8523967862129211]

Se logra un incremento en el accuracy pero un decremento considerable en el accuracy, no se tomara en cuenta esta configuracion

Se decide entonces entrenar el modelo a lo largo de varias epochs usando la mejor configuracion hasta ahora

Se juntan los datasets de test y entrenamiento para entrenar todo el modelo:

In [62]:
total_x = np.vstack((x_train, x_test))
total_y = np.hstack((y_train, y_test))

total_x.shape, total_y.shape

((20000, 34), (20000,))

In [64]:
keras.backend.clear_session()
inputs = keras.Input(shape=(max_seq,), dtype="int32")
x = keras.layers.Embedding(words+1, 256)(inputs)
x = keras.layers.Bidirectional(keras.layers.LSTM(128, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(64, return_sequences=True))(x)
x = keras.layers.Dropout(0.2)(x)
x = keras.layers.Bidirectional(keras.layers.LSTM(32))(x)
x = keras.layers.Dense(16, activation='relu')(x)
x = keras.layers.Dropout(0.2)(x)
outputs = keras.layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)

model.compile("adam", "binary_crossentropy", metrics=["accuracy", keras.metrics.Precision()])

earlyStopping = keras.callbacks.EarlyStopping(monitor="accuracy", patience=10, verbose=0, mode="max")
mcp_save = keras.callbacks.ModelCheckpoint("tweets_resulting_model.h5", save_best_only=True, monitor="accuracy", mode="max")

model.fit(total_x, total_y, batch_size=32, epochs=25,
          callbacks=[earlyStopping, mcp_save])

print("")
print("Evalaucion")
model.evaluate(x_test, y_test)

Epoch 1/25
625/625 [==============================] - 122s 179ms/step - loss: 0.4222 - accuracy: 0.7865 - precision: 0.7769
Epoch 2/25
625/625 [==============================] - 119s 190ms/step - loss: 0.1357 - accuracy: 0.9484 - precision: 0.9387
Epoch 3/25
625/625 [==============================] - 118s 188ms/step - loss: 0.0540 - accuracy: 0.9805 - precision: 0.9776
Epoch 4/25
625/625 [==============================] - 103s 165ms/step - loss: 0.0323 - accuracy: 0.9893 - precision: 0.9880
Epoch 5/25
625/625 [==============================] - 104s 166ms/step - loss: 0.0183 - accuracy: 0.9938 - precision: 0.9939
Epoch 6/25
625/625 [==============================] - 107s 171ms/step - loss: 0.0152 - accuracy: 0.9948 - precision: 0.9942
Epoch 7/25
625/625 [==============================] - 108s 174ms/step - loss: 0.0111 - accuracy: 0.9964 - precision: 0.9954
Epoch 8/25
625/625 [==============================] - 119s 191ms/step - loss: 0.0114 - accuracy: 0.9966 - precision: 0.9966
Epoch 9/

[0.011706149205565453, 0.9962499737739563, 0.9965652823448181]